In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# !pip3 install pandas

In [4]:
# try:
# #     r = requests.get('https://www.kbb.com/honda/?intent=new')
#     r = requests.get('https://www.cars.com/shopping/results/?dealer_id=&keyword=&list_price_max=&list_price_min=&makes[]=ford&maximum_distance=20&mileage_max=&page_size=20&sort=best_match_desc&stock_type=all&year_max=&year_min=&zip=20741')
# except HTTPError as e:
#     print(e)
    
# bs = BeautifulSoup(r.content, 'html.parser')
# tbl = bs.find_all('h2')
# tbl
# pbs = bs.prettify()
# pbs
df_audi = pd.read_csv('audi.csv')
df_bmw = pd.read_csv('bmw.csv')
df_ford = pd.read_csv('ford.csv')
df_hyun = pd.read_csv('hyun.csv')
df_merc = pd.read_csv('merc.csv')
df_toyo = pd.read_csv('toyo.csv')

df_master = [df_audi, df_bmw, df_ford, df_hyun, df_merc, df_toyo]
df_master = pd.concat()

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,A1,2017,12500,Manual,15735,Petrol,150,55.4,1.4
1,A6,2016,16500,Automatic,36203,Diesel,20,64.2,2.0
2,A1,2016,11000,Manual,29946,Petrol,30,55.4,1.4
3,A4,2017,16800,Automatic,25952,Diesel,145,67.3,2.0
4,A3,2019,17300,Manual,1998,Petrol,145,49.6,1.0
...,...,...,...,...,...,...,...,...,...
10663,A3,2020,16999,Manual,4018,Petrol,145,49.6,1.0
10664,A3,2020,16999,Manual,1978,Petrol,150,49.6,1.0
10665,A3,2020,17199,Manual,609,Petrol,150,49.6,1.0
10666,Q3,2017,19499,Automatic,8646,Petrol,150,47.9,1.4


In [1]:
print("test")

test
